In [1]:
from numpy import array
from numpy import asarray
from numpy import zeros
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

/Users/jmangia/anaconda2/envs/SwiftNLC/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Prepare Glove File
def readGloveFile(gloveFile):
    with open(gloveFile, 'r') as f:
        wordToGlove = {}  # map from a token (word) to a Glove embedding vector
        wordToIndex = {}  # map from a token to an index
        indexToWord = {}  # map from an index to a token 

        for line in f:
            record = line.strip().split()
            token = record[0] # take the token (word) from the text line
            wordToGlove[token] = np.array(record[1:], dtype=np.float64) # associate the Glove embedding vector to a that token (word)

        tokens = sorted(wordToGlove.keys())
        for idx, tok in enumerate(tokens):
            kerasIdx = idx + 1  # 0 is reserved for masking in Keras (see above)
            wordToIndex[tok] = kerasIdx # associate an index to a token (word)
            indexToWord[kerasIdx] = tok # associate a word to a token (word). Note: inverse of dictionary above

    return wordToIndex, indexToWord, wordToGlove

In [3]:
# Create Pretrained Keras Embedding Layer
def createPretrainedEmbeddingLayer(wordToGlove, wordToIndex, inputLength, isTrainable):
    vocabLen = len(wordToIndex) + 1  # adding 1 to account for masking
    embDim = next(iter(wordToGlove.values())).shape[0]  # works with any glove dimensions (e.g. 50)

    embeddingMatrix = np.zeros((vocabLen, embDim))  # initialize with zeros
    for word, index in wordToIndex.items():
        embeddingMatrix[index, :] = wordToGlove[word] # create embedding: word index to Glove word embedding

    embeddingLayer = Embedding(vocabLen, embDim, weights=[embeddingMatrix], input_length=inputLength, trainable=isTrainable)
    return embeddingLayer, embDim

In [4]:
# Embedding
def getEncodedDocs(docs):
    encoded_docs = []

    tokenizer = RegexpTokenizer(r'\w+')
    for doc in docs:
        encoded_doc = []
        for word in tokenizer.tokenize(doc.lower()):
            index = wordToIndex[word]
            if index is not None:
                encoded_doc.append(index)
            else:
                encoded_doc.append(0)
        encoded_docs.append(encoded_doc)

    return encoded_docs

In [5]:
# usage
max_length = 20
wordToIndex, indexToWord, wordToGlove = readGloveFile("glove/glove.6B.100d.txt")
pretrainedEmbeddingLayer, embDim = createPretrainedEmbeddingLayer(wordToGlove, wordToIndex, max_length, False)

In [6]:
# import intents file
import json
with open('PharmacyDataset.json') as json_data:
    intents = json.load(json_data)

In [7]:
classes = []
encodedUtterances = []

# loop through each sentence in our intents utterances
for intent in intents['intents']:
    classes.append(intent['intent'])
    encoded_docs = getEncodedDocs(intent['utterances'])
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    encodedUtterances.append(padded_docs)


In [8]:
currentClass = 0
train_x = []
train_y = []

for intent in classes:
    y = [0] * len(classes)
    y[currentClass] = 1

    for vector in encodedUtterances[currentClass]:
        train_x.append(vector)
        train_y.append(y)

    currentClass += 1

vectorSize = len(train_x[0])

train_X = np.array(train_x)
train_Y = np.array(train_y)

print(classes, "classes")
print(vectorSize, "vector size")
print(len(train_x), len(train_x[0]), "x")
print(len(train_y), len(train_y[0]), "y")
print(train_X.shape)
print(train_Y.shape)

([u'greeting', u'goodbye', u'thanks', u'options', u'adverse_drug', u'blood_pressure', u'blood_pressure_search', u'pharmacy_search'], 'classes')
(20, 'vector size')
(26, 20, 'x')
(26, 8, 'y')
(26, 20)
(26, 8)


# Prepare Keras / TensorFlow model

In [9]:
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Bidirectional, LSTM, GRU, TimeDistributed, Activation, Flatten, Embedding
from keras.optimizers import Adam


In [10]:
#sequence_input = Input(shape=(None, len(train_x[0])), dtype='float')
#bidiGru = Bidirectional(GRU(100))(sequence_input)
#preds = Dense(len(train_y[0]), activation='softmax')(bidiGru)
#model = Model(sequence_input, preds)
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

model = Sequential([
                    pretrainedEmbeddingLayer,
                    GRU(embDim, batch_size=1, input_shape=(None, embDim), return_sequences=True),
                    TimeDistributed(Dense(64)),
                    Activation('relu'),
                    TimeDistributed(Dense(32)),
                    Activation('relu'),
                    Flatten(),
                    Dense(len(train_y[0]), activation='softmax')
                   ])

print("model fitting - Bidirectional GRU")
model.summary()


model fitting - Bidirectional GRU
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 100)           40000100  
_________________________________________________________________
gru_1 (GRU)                  (None, 20, 100)           60300     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 64)            6464      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 64)            0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 32)            2080      
_________________________________________________________________
activation_2 (Activation)    (None, 20, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 640)  

In [11]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model

In [12]:
model.fit(train_X, train_Y, epochs=50)

Epoch 1/50
26/26 [==============================] - 0s 15ms/step - loss: 2.0886 - acc: 0.1154
Epoch 2/50
26/26 [==============================] - 0s 666us/step - loss: 2.0372 - acc: 0.2308
Epoch 3/50
26/26 [==============================] - 0s 642us/step - loss: 1.9968 - acc: 0.3077
Epoch 4/50
26/26 [==============================] - 0s 688us/step - loss: 1.9591 - acc: 0.5000
Epoch 5/50
26/26 [==============================] - 0s 681us/step - loss: 1.9234 - acc: 0.6154
Epoch 6/50
26/26 [==============================] - 0s 651us/step - loss: 1.8832 - acc: 0.6154
Epoch 7/50
26/26 [==============================] - 0s 685us/step - loss: 1.8407 - acc: 0.6923
Epoch 8/50
26/26 [==============================] - 0s 653us/step - loss: 1.7957 - acc: 0.6538
Epoch 9/50
26/26 [==============================] - 0s 655us/step - loss: 1.7473 - acc: 0.7308
Epoch 10/50
26/26 [==============================] - 0s 653us/step - loss: 1.6955 - acc: 0.7308
Epoch 11/50
26/26 [==============================]

# Export Model using CoreML Tools

In [13]:
model.save('SwiftNLCGloveRNN.h5')

In [14]:
import coremltools
coreml_model = coremltools.converters.keras.convert(model, input_names="vectors", output_names="entities")
coreml_model

0 : embedding_1_input, <keras.engine.topology.InputLayer object at 0x13f8ac090>
1 : embedding_1, <keras.layers.embeddings.Embedding object at 0x1202d1550>
2 : gru_1, <keras.layers.recurrent.GRU object at 0x125ebaad0>
3 : time_distributed_1, <keras.layers.wrappers.TimeDistributed object at 0x125ebac10>
4 : activation_1, <keras.layers.core.Activation object at 0x125ebad50>
5 : time_distributed_2, <keras.layers.wrappers.TimeDistributed object at 0x125ebae90>
6 : activation_2, <keras.layers.core.Activation object at 0x125ebaed0>
7 : flatten_1, <keras.layers.core.Flatten object at 0x125ebaf10>
8 : dense_3, <keras.layers.core.Dense object at 0x125ebaf90>
9 : dense_3__activation__, <keras.layers.core.Activation object at 0x122091e50>


input {
  name: "vectors"
  type {
    multiArrayType {
      shape: 1
      dataType: DOUBLE
    }
  }
}
input {
  name: "gru_1_h_in"
  type {
    multiArrayType {
      shape: 100
      dataType: DOUBLE
    }
    isOptional: true
  }
}
output {
  name: "entities"
  type {
    multiArrayType {
      shape: 8
      dataType: DOUBLE
    }
  }
}
output {
  name: "gru_1_h_out"
  type {
    multiArrayType {
      shape: 100
      dataType: DOUBLE
    }
  }
}

In [15]:
coreml_model.save('SwiftNLCGloveRNN.mlmodel')